In [1]:
!pip3 install pyarxiv
!pip3 install pybtex
!pip3 install dropbox
!pip3 install tqdm
!pip3 install pycolors2
!pip3 install pdf2image
!pip3 install Unidecode


[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
from bib_handling_code.processbib import read_bibfile
from bib_handling_code.processbib import save_to_file
import requests
import pandas as pd 
from get_biblatex import GetBiblatex
import string

In [35]:
path_diag_bib = os.path.join('script_data/', 'diag_ss.bib')
diag_bib_raw = read_bibfile(None, path_diag_bib) # I changed the code in such a way that IF I give a second argument, it uses the second argument as a full path

In [36]:
staff_id_dict = {'Bram van Ginneken': [8038506, 123637526],
'Francesco Ciompi': [143613202],
'Alessa Hering': [153744566],
'Henkjan Huisman': [34754023],
'Colin Jacobs': [2895994],
'Peter Koopmans': [34726383],
'Jeroen van der Laak': [145441238, 145388932],
'Geert Litjens': [145959882],
'James Meakin': [4960344],
'Keelin Murphy': [35730362],
'Ajay Patel': [2109170880, 2116215861],
'Cornelia Schaefer-Prokop': [1419819133, 1445069528, 1400632685],
'Matthieu Rutten': [2074975080, 2156546],
'Jos Thannhauser': [5752941],
"Bram Platel" : [1798137], 
"Nico Karssemeijer" : [1745574], 
"Clarisa Sanchez" : [144085811, 32187701], 
"Nikolas Lessman" : [2913408], 
"Jonas Teuwen" : [32649341, 119024451], 
"Rashindra Manniesing" : [2657081]}

In [37]:
staff_year_dict = {
'Bram van Ginneken':  {'start' : 1996, 'end': 9999},
'Francesco Ciompi':  {'start' : 2013, 'end': 9999},
'Alessa Hering':  {'start' : 2018, 'end': 9999},
'Henkjan Huisman':  {'start' : 1992, 'end': 9999},
'Colin Jacobs':  {'start' : 2010, 'end': 9999},
'Peter Koopmans':  {'start' : 2022, 'end': 9999},
'Jeroen van der Laak':  {'start' : 1991, 'end': 9999},
'Geert Litjens':  {'start' : 2016, 'end': 9999},
'James Meakin':  {'start' : 2017, 'end': 9999},
'Keelin Murphy':  {'start' : 2018, 'end': 9999},
'Ajay Patel':  {'start' : 2015, 'end': 9999},
'Cornelia Schaefer-Prokop':  {'start' : 2010, 'end': 9999},
'Matthieu Rutten':  {'start' : 2019, 'end': 9999},
'Jos Thannhauser': {'start' : 2022, 'end': 9999},
"Bram Platel" : {'start' : 2010,  'end' : 2019},
"Nico Karssemeijer" : {'start' : 1989, 'end' : 2022}, 
"Clarisa Sanchez" : {'start' : 2008, 'end' : 2021}, 
"Nikolas Lessman" : {'start' : 2019, 'end' : 2022}, 
"Jonas Teuwen" : {'start' : 2017, 'end' : 2020}, 
"Rashindra Manniesing" : {'start' : 2010, 'end' : 2021}
}

In [38]:
def find_new_ssids():
    staff_dict = {key: {'ids': staff_id_dict[key], 'years': staff_year_dict[key]} for key in staff_id_dict}
    all_staff_id_ss_data = []

    # get data for staff memb|er
    for idx, (staff_name, values) in enumerate(staff_dict.items()):
        staff_ids = values['ids']
        staff_start = values['years']['start']
        staff_end = values['years']['end']
        print(f'[{idx+1}/{len(staff_dict.items())}]: {staff_name}')
        for staff_id in staff_ids:
            print('\t\t', staff_id)
            staff_id_ss_data = []
            # get semantic scolar data
            url=f'https://api.semanticscholar.org/graph/v1/author/{staff_id}/papers?fields=year,title,authors,externalIds,citationCount&limit=500'
            r = requests.get(url)
            ss_staff_data = r.json()['data']

            # loop over semantic scolar data entries
            for ss_staff_entry in ss_staff_data:
                ss_id = None
                ss_title = None
                ss_doi = None
                ss_citations = None
                ss_year = None

                if 'paperId' in ss_staff_entry:
                    ss_id = ss_staff_entry['paperId']
                if 'title' in ss_staff_entry:
                    ss_title = ss_staff_entry['title']
                if 'DOI' in ss_staff_entry['externalIds']:
                    ss_doi = ss_staff_entry['externalIds']['DOI']
                if 'citationCount' in ss_staff_entry:
                    ss_citations = ss_staff_entry['citationCount']
                if 'year' in ss_staff_entry:
                    ss_year = ss_staff_entry['year']

                if ss_year != None:
                    if not staff_start <= ss_year <= staff_end:
                    # probably doesnt belong to DIAG, still captured via another staff member if also in the same paper
                        continue

                staff_id_ss_data.append([staff_id, staff_name, staff_start, staff_end, ss_year, ss_id, ss_title, ss_doi, ss_citations])
            all_staff_id_ss_data.extend(staff_id_ss_data)    
    ss_columns = ['staff_id', 'staff_name', 'staff_from', 'staff_till', 'ss_year', 'ss_id', 'title', 'doi', 'ss_citations']
    df_all_staff_id_ss_data = pd.DataFrame(all_staff_id_ss_data, columns=ss_columns)
    print('DONE')
    return df_all_staff_id_ss_data, df_all_staff_id_ss_data['ss_id'].tolist()

In [39]:
url=f'https://api.semanticscholar.org/graph/v1/author/{143613202}/papers?fields=year,title,authors,externalIds,citationCount&limit=500'
r = requests.get(url)
ss_staff_data = r.json()['data']

In [40]:
def return_current_ssids(bib_file):
    ss_ids=[]
    for entry in bib_file:
        if entry.type == 'string':
            continue
        if 'ss_id' in entry.fields:
            ss_ids.append(entry.fields['ss_id'].translate(str.maketrans('', '', string.punctuation)))
    return ss_ids

In [41]:
df, new_ss_ids = find_new_ssids()
current_ss_ids=return_current_ssids(diag_bib_raw)

[1/20]: Bram van Ginneken
		 8038506
		 123637526
[2/20]: Francesco Ciompi
		 143613202
[3/20]: Alessa Hering
		 153744566
[4/20]: Henkjan Huisman
		 34754023
[5/20]: Colin Jacobs
		 2895994
[6/20]: Peter Koopmans
		 34726383
[7/20]: Jeroen van der Laak
		 145441238
		 145388932
[8/20]: Geert Litjens
		 145959882
[9/20]: James Meakin
		 4960344
[10/20]: Keelin Murphy
		 35730362
[11/20]: Ajay Patel
		 2109170880
		 2116215861
[12/20]: Cornelia Schaefer-Prokop
		 1419819133
		 1445069528
		 1400632685
[13/20]: Matthieu Rutten
		 2074975080
		 2156546
[14/20]: Jos Thannhauser
		 5752941
[15/20]: Bram Platel
		 1798137
[16/20]: Nico Karssemeijer
		 1745574
[17/20]: Clarisa Sanchez
		 144085811
		 32187701
[18/20]: Nikolas Lessman
		 2913408
[19/20]: Jonas Teuwen
		 32649341
		 119024451
[20/20]: Rashindra Manniesing
		 2657081
DONE


In [42]:
print(f"Number of current ss_ids in diag_bib: {len(current_ss_ids)}")
print(f"Number of new ss_ids: {len(new_ss_ids)}")


Number of current ss_ids in diag_bib: 917
Number of new ss_ids: 2200


## ss_ids not in the diag_bib but not in found by our staff member search

In [43]:
set(current_ss_ids) - set(new_ss_ids)

{'0257287f6628466cb01815358a9955962faf43af',
 '057100455efca559a6d38d31ed35cf56bdefc958',
 '0ab99aa04e3a8340a7552355fb547374a5604b24',
 '1a3c9f430cf163a3c2decdc9f04dae285739ace6',
 '3d01a44659fce1fb0134b6d8baabbee72583d3fe',
 '6493a6974ae6cb8e4188894fb9e2b5a6107fe424',
 '7cbf61f2d7b1fae068e3b815ebcb09e09555cd50',
 '9ca57befc51aee0d4bf6f19e951a776e4d1c9d82',
 'efeaeef3e03c7e488095f3a38809d82b5f1766ac'}

There are 1631 unique semantic scholar ids found. The current bib file has 915 unique semantic scholar ids.
The intersection is 910 = that means that there are 5 semantic scholar ids in the current bib that are not found from the new search. Why is that?

In [44]:
found_items=set(new_ss_ids)-set(current_ss_ids)
print(f"Number of newly found items: {len(found_items)}")

Number of newly found items: 721


In [45]:
new_items = df[df['ss_id'].isin(found_items)]

In [46]:
new_items = new_items[new_items['ss_year']>=2015]
new_items = new_items.loc[new_items['doi'].notna()]
new_items = new_items.drop_duplicates(subset='doi', keep='first')

In [47]:
new_items

,staff_id,staff_name,staff_from,staff_till,ss_year,ss_id,title,doi,ss_citations
2,8038506,Bram van Ginneken,1996,9999,2023.0,3c7c167e0619911a29ce6082372100fbbc5ca7af,Emphysema subtyping on thoracic computed tomog...,10.1038/s41598-023-40116-6,0
3,8038506,Bram van Ginneken,1996,9999,2023.0,4bcd672218ecec70473c84f6f1cc52c64031f3e5,Continual learning strategies for cancer-indep...,10.1016/j.media.2023.102755,3
4,8038506,Bram van Ginneken,1996,9999,2023.0,7981606bf8110ec6cc64baa22d694096f7862939,AIROGS: Artificial Intelligence for RObust Gla...,10.48550/arXiv.2302.01738,4
5,8038506,Bram van Ginneken,1996,9999,2023.0,9ddb2f47695191553a3623ac33eddeb9c7e416cd,The STOIC2021 COVID-19 AI challenge: applying ...,10.48550/arXiv.2306.10484,1
7,8038506,Bram van Ginneken,1996,9999,2023.0,be95c8bbd8a4297d620b1c2644cf2a898603e355,Kidney abnormality segmentation in thorax-abdo...,10.48550/arXiv.2309.03383,0
...,...,...,...,...,...,...,...,...,...
2137,2657081,Rashindra Manniesing,2010,2021,2021.0,4202f56b0e22eedd6921d9b26519bc72f89cf4fd,Correction to: Incorporating radiomics into cl...,10.1007/s00330-021-07721-3,4
2139,2657081,Rashindra Manniesing,2010,2021,2021.0,571285fab9ced6a4d0bf3abd5f886d9a64e1790c,Traumatic Cerebral Microbleeds in the Subacute...,10.3174/ajnr.A7028,2
2140,2657081,Rashindra Manniesing,2010,2021,2021.0,92a2cb9804d3599cc1e8e81c512d0203c6d10da7,The radiological interpretation of possible mi...,10.1007/s00234-021-02839-z,2
2141,2657081,Rashindra Manniesing,2010,2021,2021.0,c859bd469080b82dc14db62e78d65ef5b5ffa686,Incorporating radiomics into clinical trials: ...,10.1007/s00330-020-07598-8,44


In [48]:
dois = new_items['doi'].tolist()

In [49]:
ss_ids = new_items['ss_id'].tolist()

**Here: remove blacklist items**

Check dois: the first one refers to this publication https://www.semanticscholar.org/paper/Continual-learning-strategies-for-detection-of-node-B%C3%A1ndi-Balkenhol/4bcd672218ecec70473c84f6f1cc52c64031f3e5, it's already in diag.bib and should have a ss_id associated (it seems to be a one-to-one match). Why is it detected as a new item?

In [32]:
!pip3 install semanticscholar

In [33]:
# Code to get citations from semantic scholar. If there are multiple ss_ids, we should get the number of citations for each of them and sum the two (or more?) values. 
def get_citations(ss_id):
    from semanticscholar import SemanticScholar
    sch = SemanticScholar()
    paper = sch.get_paper(ss_id)
    return len(paper['citations'])

In [45]:
# Read the diag.bib file
cwd = os.getcwd()
parent_directory = os.path.abspath(os.path.join(cwd, ".."))
diag_bib_path = os.path.join(parent_directory, 'diag.bib')

with open(diag_bib_path, encoding="utf8") as bibtex_file:
    diag_bib = bibtex_file.read()

for i, el in enumerate(dois): #Make sure to only use unique dois
    citations = 0
    if isinstance(ss_ids[i], list):
        for ss_id in ss_ids[i]:
            citations += get_citations(ss_id)
    else:
        citations = get_citations(ss_ids[i])
    # Update Dre's code and add an extra argument for citations so that it uses the correct number when creating the bib item? 
    reader = GetBiblatex(doi=el, diag_bib=diag_bib, num_citations=citations)
    bibtext = reader.get_bib_text()
    # Code to add item to diag.bib ?
    if bibtext != 'empty':
        diag_bib = diag_bib + bibtext
